# Arbol de decision
En este notebook vamos a entrenar y explorar un arbol de decision.

In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing, tree
from ipywidgets import Button, IntSlider, interactive

from preprocessing import cargarDatasets
from preprocessing import prepararSetDeEntrenamiento
from preprocessing import prepararSetDeValidacion
from preprocessing import ingenieriaDeFeauturesArboles1

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split,GridSearchCV

## Carga y preprocesamiento de los datos
Cargamos los datasets y los preparamos para usar.

In [15]:
train_df,final_df = cargarDatasets()
train_df = prepararSetDeEntrenamiento(train_df)
final_df = prepararSetDeValidacion(final_df)

Ahora que ya tenemos los datasets cargados, **le aplicamos al dataset de entrenamiento distintos preprocesamientos, y luego lo particionamos usando k-fold cross validation.** a chequear esto ultimo

In [16]:
X,y,df,y_encoder = ingenieriaDeFeauturesArboles1(train_df)

* ingenieria de feautures (2 formas)
* dividimos cada set en 3.
* entrenamos (2 arboles)
* metricas

fit: ajusta el modelo al set de entrenamiento
predict: que agarre un ejemplo y haga una prediccion (predice la clase a la que pertenece)


hiperparametros: quiero encontrar el mejor modelo para este dataset. Son parametros que el modelo no aprende, hay que decirselos.
-> random state: quiero que si un valor es random, que sea reproducible. Que el modelo vuelva a dar lo mismo cada vez que lo corro (que sea determinista).
->split: cuantos feautures miro para ir dividiendo.

In [5]:
arbol_1 = tree.DecisionTreeClassifier()
params = {'max_depth': np.arange(1, 31), 'min_samples_leaf': np.arange(1, 16)}
gscv = GridSearchCV(
    arbol_1, params, scoring='accuracy', n_jobs=-1, cv=5, return_train_score=True
).fit(X, y)

In [8]:
print(f"Best score: {gscv.best_score_}")
print(f"Best params {gscv.best_params_}")

Best score: 0.8594946423539238
Best params {'max_depth': 11, 'min_samples_leaf': 9}


In [11]:
params = {
    'max_depth': np.arange(1, 10),
    'min_samples_leaf': np.arange(1, 10),
    "criterion": ["gini", "entropy"],
}

arbol_2 = tree.DecisionTreeClassifier()
gscv = GridSearchCV(
    arbol_1, params, scoring='accuracy', n_jobs=-1, cv=5, return_train_score=True
).fit(X, y)

In [13]:
print(f"Best score: {gscv.best_score_}")
print(f"Best params {gscv.best_params_}")

Best score: 0.8578975718945779
Best params {'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 4}
